In [1]:
import os
import subprocess
import dotenv
import time
import glob
import hashlib
import shutil
import logging
import torch

config_test_patplusplus = "config/UrbanElementsReID_test_patplusplus.yml"
track = "assets/models/PAT_plusplus/track.txt"
submission_file_name = "assets/models/PAT_plusplus/track_submission.csv"

competition_name = "urban-reid-challenge"
submission_message = f"PAT++, simple matching"

################ Probably nothing has to be modified from now on ################

assert dotenv.load_dotenv('../../.env')
assert os.getenv('KAGGLE_USERNAME')

from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

logging.root.setLevel(logging.INFO)

if not torch.cuda.is_available():
    logging.warning("Where is your GPU dude?")

In [2]:
!python update.py --config_file {config_test_patplusplus} --track {track}

2025-05-12 00:06:24,414 PAT INFO: Namespace(config_file='config/UrbanElementsReID_test_patplusplus.yml', opts=[], track='assets/models/PAT_plusplus/track.txt')
2025-05-12 00:06:24,414 PAT INFO: Loaded configuration file config/UrbanElementsReID_test_patplusplus.yml
2025-05-12 00:06:24,415 PAT INFO: 
MODEL:
  PRETRAIN_CHOICE: 'imagenet'
  #PRETRAIN_PATH: "../../.cache/torch/hub/checkpoints" # root of pretrain path
  PRETRAIN_PATH: "assets/models/PAT_r" 
  METRIC_LOSS_TYPE: 'triplet'
  IF_LABELSMOOTH: 'on'
  IF_WITH_CENTER: 'no'
  NAME: 'part_attention_vit'
  NO_MARGIN: True
  DEVICE_ID: ('0')
  TRANSFORMER_TYPE: 'vit_base_patch16_224_TransReID'
  STRIDE_SIZE: [16, 16]

INPUT:
  SIZE_TRAIN: [256,128]
  SIZE_TEST: [256,128]
  REA:
    ENABLED: False
  PIXEL_MEAN: [0.5, 0.5, 0.5]
  PIXEL_STD: [0.5, 0.5, 0.5]
  LGT: # Local Grayscale Transfomation
    DO_LGT: True
    PROB: 0.5

DATASETS:
  TRAIN: ('UrbanElementsReID',)
  TEST: ('UrbanElementsReID_test',)
  #ROOT_DIR: ('../../data') # root 

In [3]:
!ls -lah assets/models/PAT_plusplus/

total 331M
drwxr-xr-x.  2 benle1 benle1 4.0K May 11 23:51 .
drwxr-xr-x. 36 benle1 benle1 4.0K May 11 21:01 ..
-rw-r--r--.  1 benle1 benle1 331M May 11 23:51 part_attention_vit_60.pth
-rw-r--r--.  1 benle1 benle1 5.1K May 12 00:06 test_log.txt
-rw-r--r--.  1 benle1 benle1 132K May 12 00:06 track.txt
-rw-r--r--.  1 benle1 benle1 136K May 12 00:06 track_submission.csv
-rw-r--r--.  1 benle1 benle1  37K May 11 23:51 train_log.txt


# Submit

In [4]:
submission_message += f"; commit_hash: {subprocess.check_output(['git', 'rev-parse', 'HEAD']).decode('ascii').strip()}"
logging.info(f'Submitting with message "{submission_message}"')

api.competition_submit(submission_file_name, submission_message, competition_name)    

INFO:root:Submitting with message "PAT++, simple matching; commit_hash: 9b3d05be76bd8535482da8c6d8d63b134e3a8468"
100%|██████████| 135k/135k [00:00<00:00, 181kB/s]


{"message": "Successfully submitted to Urban Elements ReID Challenge", "ref": 44686968}